In [ ]:
!pip install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.6/164.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import os
import zipfile
from hmmlearn import hmm
import matplotlib.pyplot as plt

In [ ]:
# Parameters
patients = 5
channels_per_series = 750
time_points = 10240  # 20 seconds at 512 Hz
sampling_rate = 512  # Hz

# Time array
time = np.linspace(0, 20, time_points)  # 20 seconds

# Function to generate synthetic signals
def generate_focal_signal(time, noise_level=0.5):
    """Generate synthetic focal signal (seizure-like)."""
    base_signal = 3 * np.sin(2 * np.pi * 10 * time)  # High frequency component (10 Hz)
    noise = noise_level * np.random.normal(size=len(time))  # Random noise
    return base_signal + noise

def generate_nonfocal_signal(time, noise_level=0.2):
    """Generate synthetic nonfocal signal (healthy-like)."""
    base_signal = np.sin(2 * np.pi * 2 * time)  # Low frequency component (2 Hz)
    noise = noise_level * np.random.normal(size=len(time))  # Random noise
    return base_signal + noise

# Generate synthetic data
synthetic_data = {}

for patient in range(1, patients + 1):
    patient_data = {"F_series": [], "N_series": []}

    for channel in range(channels_per_series):
        # Focal series (F-series)
        focal_x = generate_focal_signal(time)
        focal_y = focal_x * 0.8 + np.random.normal(0, 0.3, size=len(time))
        patient_data["F_series"].append((focal_x, focal_y))

        # Nonfocal series (N-series)
        nonfocal_x = generate_nonfocal_signal(time)
        nonfocal_y = nonfocal_x * 0.9 + np.random.normal(0, 0.1, size=len(time))
        patient_data["N_series"].append((nonfocal_x, nonfocal_y))

    synthetic_data[f"Patient_{patient}"] = patient_data

# Data is stored in the `synthetic_data` dictionary


In [ ]:
# Get data for Patient 1
patient_1_data = synthetic_data["Patient_1"]

# Get focal signal for the first channel of Patient 1
focal_x, focal_y = patient_1_data["F_series"][0]

# Get nonfocal signal for the first channel of Patient 1
nonfocal_x, nonfocal_y = patient_1_data["N_series"][0]

In [ ]:
# Initialize an array to hold all observed data from all patients
all_observed_data = []

# Loop through the synthetic data for 5 patients
for patient_idx in range(1, patients + 1):  # Iterating through 5 patients
    # Get the F-series (focal signals) for each patient
    patient_data = synthetic_data[f"Patient_{patient_idx}"]

    # For simplicity, we'll use the 'focal_x' signal from each channel
    # Extract 'focal_x' from each channel in the patient's F-series
    # 'patient_data["F_series"]' is a list of tuples: (focal_x, focal_y)
    for channel_data in patient_data["F_series"]:
        focal_x = channel_data[0]  # Extract focal_x (first element of the tuple)
        all_observed_data.append(focal_x)  # Add focal_x to the observed data list

# Combine the data from all patients into a single array for fitting
combined_data = np.concatenate(all_observed_data)  # Concatenate all patient data into one array

# Reshape the data to fit the HMM (HMM expects 2D data: samples x features)
combined_data_reshaped = combined_data.reshape(-1, 1)

# Fit the HMM model
model = hmm.GaussianHMM(n_components=3, covariance_type="diag", n_iter=1000)

# Fit the model to the combined data
model.fit(combined_data_reshaped)

# Now, the model is trained and can be used for decoding sequences or inference
print("HMM training completed.")


KeyboardInterrupt: 